In [7]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [8]:
class SpotifyAPI(object):
    access_token=None
    access_token_expiry = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded STRING
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception ("You must set client Id and client secret ")
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        
        return client_creds_b64.decode()
        
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
    "Authorization": f"Basic {client_creds_b64}" 
}
    
    def get_token_data(self):
        return {
    "grant_type":"client_credentials"
}
    
    def perform_auth(self):
        
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url,data=token_data,headers=token_headers)
        
        if r.status_code not in range(200,299):
            raise Exception("Authentication failed")
        
        data = r.json()
        now = datetime.datetime.now()
        self.access_token = data['access_token']
        expires_in = data['expires_in'] #in seconds
        expiry = now+datetime.timedelta(seconds=expires_in)
        self.access_token_expiry = expiry
        self.access_token_did_expire = expiry<now
        
        return True
     
    def get_access_token(self):

        token = self.access_token
        expires = self.access_token_expiry
        now = datetime.datetime.now()
        if expires<now or token==None:
            self.perform_auth()
            return self.get_access_token()
        
        return token
    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        
        headers = { 
            'Authorization':f'Bearer {access_token}'
        }
        return headers
        
    def get_resource(self, lookup_id, resource_type='albums',version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_headers()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
        
    def get_album(self,_id):
        return self.get_resource(_id,resource_type="albums")
    
    def get_artist(self,_id):
        return self.get_resource(_id,resource_type="artists")
    
    def search(self,query,search_type='artist'):
        
        headers = self.get_resource_headers()
        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({"q":query,"type":search_type.lower()})

        lookup_url = f'{endpoint}?{data}'
        print(lookup_url)
        r=requests.get(lookup_url, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        
        return r.json()
    
    

## Using the above class to do some stuff

In [9]:
import os

client_id = os.environ['SPOTIFY_CID']
client_secret = os.environ['SPOTIFY_CS']

In [10]:
spotify = SpotifyAPI(client_id,client_secret)

In [13]:
spotify.search("Making Music",search_type="album")

https://api.spotify.com/v1/search?q=Making+Music&type=album


{'albums': {'href': 'https://api.spotify.com/v1/search?query=Making+Music&type=album&offset=0&limit=20',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1ThoqLcyIYvZn7iWbj8fsj'},
      'href': 'https://api.spotify.com/v1/artists/1ThoqLcyIYvZn7iWbj8fsj',
      'id': '1ThoqLcyIYvZn7iWbj8fsj',
      'name': 'Bill Withers',
      'type': 'artist',
      'uri': 'spotify:artist:1ThoqLcyIYvZn7iWbj8fsj'}],
    'available_markets': ['AD',
     'AE',
     'AL',
     'AR',
     'AT',
     'AU',
     'BA',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HR',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JO',
     'JP',
     'KW',
    

In [12]:
spotify.get_artist('2MphKCPv2wbV61u03gKP4i')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2MphKCPv2wbV61u03gKP4i'},
 'followers': {'href': None, 'total': 166},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/2MphKCPv2wbV61u03gKP4i',
 'id': '2MphKCPv2wbV61u03gKP4i',
 'images': [],
 'name': 'Zakir Hussain, Krishna Bhatt',
 'popularity': 4,
 'type': 'artist',
 'uri': 'spotify:artist:2MphKCPv2wbV61u03gKP4i'}

In [14]:
spotify.get_album('5CfytRni4jhBrwM2MdPg1N')

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1ThoqLcyIYvZn7iWbj8fsj'},
   'href': 'https://api.spotify.com/v1/artists/1ThoqLcyIYvZn7iWbj8fsj',
   'id': '1ThoqLcyIYvZn7iWbj8fsj',
   'name': 'Bill Withers',
   'type': 'artist',
   'uri': 'spotify:artist:1ThoqLcyIYvZn7iWbj8fsj'}],
 'available_markets': ['AD',
  'AE',
  'AL',
  'AR',
  'AT',
  'AU',
  'BA',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'BY',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HR',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'KZ',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV',
  'MA',
  'MC',
  'MD',
  'ME',
  'MK',
  'MT',
  'MX',
  'MY',
  'NI',
  'NL',
  'NO',
  'NZ',
  'OM',
  'PA',
  'PE',
  'PH',
  'PL',
  'PS',
  'PT',
  'PY',
  'QA',
  'RO',
  'RS',
  'RU',
  'SA',
  'SE',
  'SG',
  'SI',
  'SK